In [5]:
import pandas as pd
import numpy as np
import os

In [16]:
data_dir_base = "/Users/hn/Documents/01_research_data/RangeLand/Data/"
Shannon_data_dir = data_dir_base + "Shannon_Data/"
Min_data_dir_base = data_dir_base + "Min_Data/"
USDA_data_dir = data_dir_base + "/NASS_downloads/"
reOrganized_dir = data_dir_base + "reOrganized/"
os.makedirs(reOrganized_dir, exist_ok=True)

import sys
sys.path.append("/Users/hn/Documents/00_GitHub/Rangeland/Python_Codes/")
import rangeland_core as rc

In [8]:
cattle_inventory = pd.read_csv(USDA_data_dir + "/cow_inventory_Qs/"+ "Q4.csv")
cattle_inventory.rename(columns=lambda x: x.lower().replace(' ', '_'), inplace=True) 
cattle_inventory.head(2)

,program,year,period,week_ending,geo_level,state,state_ansi,ag_district,ag_district_code,county,...,zip_code,region,watershed_code,watershed,commodity,data_item,domain,domain_category,value,cv_(%)
0,CENSUS,2017,END OF DEC,NaN,COUNTY,ALABAMA,1,BLACK BELT,40,AUTAUGA,...,NaN,NaN,0,NaN,CATTLE,"CATTLE, COWS, BEEF - INVENTORY",TOTAL,NOT SPECIFIED,"8,678",20.9
1,CENSUS,2017,END OF DEC,NaN,COUNTY,ALABAMA,1,BLACK BELT,40,BULLOCK,...,NaN,NaN,0,NaN,CATTLE,"CATTLE, COWS, BEEF - INVENTORY",TOTAL,NOT SPECIFIED,(D),(D)


In [9]:
bad_cols  = ["watershed", "watershed_code", 
             "domain", "domain_category", 
             "region", "period",
             "week_ending", "zip_code", "program", "geo_level"]


meta_cols = ["state", "county", "county_ansi", "state_ansi", "ag_district_code"]

In [10]:
cattle_inventory.drop(bad_cols, axis="columns", inplace=True)

In [11]:
cattle_inventory['county_ansi'].fillna(666, inplace=True)

cattle_inventory["state_ansi"] = cattle_inventory["state_ansi"].astype('int32')
cattle_inventory["county_ansi"] = cattle_inventory["county_ansi"].astype('int32')

cattle_inventory["state_ansi"] = cattle_inventory["state_ansi"].astype('str')
cattle_inventory["county_ansi"] = cattle_inventory["county_ansi"].astype('str')

cattle_inventory.state = cattle_inventory.state.str.title()
cattle_inventory.county = cattle_inventory.county.str.title()

for idx in cattle_inventory.index:
    if len(cattle_inventory.loc[idx, "state_ansi"]) == 1:
        cattle_inventory.loc[idx, "state_ansi"] = "0" + cattle_inventory.loc[idx, "state_ansi"]
        
    if len(cattle_inventory.loc[idx, "county_ansi"]) == 1:
        cattle_inventory.loc[idx, "county_ansi"] = "00" + cattle_inventory.loc[idx, "county_ansi"]
    elif len(cattle_inventory.loc[idx, "county_ansi"]) == 2:
        cattle_inventory.loc[idx, "county_ansi"] = "0" + cattle_inventory.loc[idx, "county_ansi"]
        
        
cattle_inventory["county_fips"] = cattle_inventory["state_ansi"] + cattle_inventory["county_ansi"]
cattle_inventory[["state", "county", "state_ansi", "county_ansi", "county_fips"]].head(5)

,state,county,state_ansi,county_ansi,county_fips
0,Alabama,Autauga,01,001,01001
1,Alabama,Bullock,01,011,01011
2,Alabama,Dallas,01,047,01047
3,Alabama,Elmore,01,051,01051
4,Alabama,Greene,01,063,01063


In [14]:
cattle_inventory.head(2)

,year,state,state_ansi,ag_district,ag_district_code,county,county_ansi,commodity,data_item,value,cv_(%),county_fips
0,2017,Alabama,01,BLACK BELT,40,Autauga,001,CATTLE,"CATTLE, COWS, BEEF - INVENTORY","8,678",20.9,01001
1,2017,Alabama,01,BLACK BELT,40,Bullock,011,CATTLE,"CATTLE, COWS, BEEF - INVENTORY",(D),(D),01011


In [12]:
len(cattle_inventory.state.unique())

25

In [13]:
len(cattle_inventory.county_fips.unique())

2101

In [18]:
cattle_inventory.rename(
    columns={"value": "cattle_cow_beef_inventory", "cv_(%)": "cattle_cow_beef_inventory_cv_(%)"},
    inplace=True,
)

print (cattle_inventory.shape)
cattle_inventory = rc.clean_census(df=cattle_inventory, col_="cattle_cow_beef_inventory")
print (cattle_inventory.shape)

(10484, 12)
(7900, 12)


In [19]:
len(cattle_inventory.county_fips.unique())

2052

In [21]:
NPP = pd.read_csv(Min_data_dir_base + "county_annual_MODIS_NPP.csv")
NPP.rename(columns={"NPP": "modis_npp"}, inplace=True)

NPP = rc.correct_Mins_FIPS(df=NPP, col_="county")
NPP.rename(columns={"county": "county_fips"}, inplace=True)

NPP.head(2)

,year,county_fips,modis_npp
0,2001,01003,0.926441
1,2001,01005,0.629133


In [23]:
len(NPP.county_fips.unique())

2256

In [24]:
NPP.year.min()

2001

In [26]:
sorted(cattle_inventory.year.unique())

[1997, 2002, 2007, 2012, 2017]

In [27]:
NPP = NPP[NPP.year >= 2002]
cattle_inventory = cattle_inventory[cattle_inventory.year >= 2002]

In [29]:
len(cattle_inventory.county_fips.unique())

2000

In [30]:
NPP_counties = set(NPP.county_fips)
cattle_inventory_counties = set(cattle_inventory.county_fips)

# intersetion of counties in common between NPP and cow_inventory

In [33]:
intersected_counties = NPP_counties.intersection(cattle_inventory_counties)
len(intersected_counties)

1489

# Counties for which we have data for all years

In [34]:
NPP = NPP[NPP.county_fips.isin(intersected_counties)]
cattle_inventory = cattle_inventory[cattle_inventory.county_fips.isin(intersected_counties)]
cattle_inventory = cattle_inventory[["year", "county_fips", "cattle_cow_beef_inventory"]]

In [45]:
NPP.reset_index(drop=True, inplace=True)
cattle_inventory.reset_index(drop=True, inplace=True)
cattle_inventory.head(2)

,year,county_fips,cattle_cow_beef_inventory
0,2017,01003,11212.0
1,2017,01035,6851.0


In [46]:
NPP.head(2)

,year,county_fips,modis_npp
0,2002,01003,0.826076
1,2002,01005,0.484333


In [49]:
NPP_inventory = pd.merge(NPP, cattle_inventory, on = ["county_fips", "year"], how = "left")

print (len(NPP_inventory.county_fips.unique()))
NPP_inventory.dropna(how="any", inplace=True)

print (len(NPP_inventory.county_fips.unique()))

In [59]:
full_counties = []
for a_county in NPP_inventory.county_fips.unique():
    df = NPP_inventory[NPP_inventory.county_fips == a_county]
    if len(df.year.unique()) == 4:
        full_counties = full_counties + [a_county]
        

In [60]:
len(full_counties)

695